<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#number-of-tokens-in-all-the-sequences." data-toc-modified-id="number-of-tokens-in-all-the-sequences.-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>number of tokens in all the sequences.</a></span></li></ul></div>

In [1]:
from tensorflow.keras.layers import Embedding,Flatten,Dense,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
import os
import numpy as np

In [2]:
!ls  './data/aclImdb'

README
imdb.vocab
imdbEr.txt
test
train


In [3]:
data_dir=os.path.dirname('./data/aclImdb/')
train_dir = os.path.join(data_dir, 'train')
labels=[]
train_texts=[]
for label_type in ['neg','pos']:
    dir_name=os.path.join(train_dir,label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:]=='.txt':
            file=open(os.path.join(dir_name,fname),'r',encoding='utf-8')
            read_file=file.read()
            train_texts.append(read_file)
            file.close()
            if label_type=='neg':
                labels.append(0)
            else:
                 labels.append(1)     


In [4]:
train_texts[1]

"Airport '77 starts as a brand new luxury 747 plane is loaded up with valuable paintings & such belonging to rich businessman Philip Stevens (James Stewart) who is flying them & a bunch of VIP's to his estate in preparation of it being opened to the public as a museum, also on board is Stevens daughter Julie (Kathleen Quinlan) & her son. The luxury jetliner takes off as planned but mid-air the plane is hi-jacked by the co-pilot Chambers (Robert Foxworth) & his two accomplice's Banker (Monte Markham) & Wilson (Michael Pataki) who knock the passengers & crew out with sleeping gas, they plan to steal the valuable cargo & land on a disused plane strip on an isolated island but while making his descent Chambers almost hits an oil rig in the Ocean & loses control of the plane sending it crashing into the sea where it sinks to the bottom right bang in the middle of the Bermuda Triangle. With air in short supply, water leaking in & having flown over 200 miles off course the problems mount for 

In [5]:
labels[80:100]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [6]:
# keep the 15000 most common words
max_features=15000
embedding_dim=16
tokenizer=Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train_texts)
def preprocess_text(text):
    sequence=tokenizer.texts_to_sequences(text)
    return sequence

In [7]:
sequence=preprocess_text(train_texts)

## number of tokens in all the sequences.

In [8]:
num_tokens=[len(tokens) for tokens in sequence]
max_tokens=int(np.mean(num_tokens))
max_tokens

228

In [9]:
np.max(num_tokens)

2307

In [10]:
train_sequence_pad=pad_sequences(sequence,maxlen=max_tokens)

In [11]:
model=Sequential()
model.add(Embedding(input_dim=max_features,output_dim=embedding_dim,input_length=max_tokens))
model.add(Flatten())
model.add(Dense(14,activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 228, 16)           240000    
_________________________________________________________________
flatten (Flatten)            (None, 3648)              0         
_________________________________________________________________
dense (Dense)                (None, 14)                51086     
_________________________________________________________________
dropout (Dropout)            (None, 14)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 15        
Total params: 291,101
Trainable params: 291,101
Non-trainable params: 0
_________________________________________________________________


In [12]:
labels=np.array(labels)

In [13]:
train_sequence_pad

array([[   0,    0,    0, ...,   27,  107, 3348],
       [ 747,  286,  531, ...,    1, 4517, 5457],
       [   0,    0,    0, ..., 1906,   15,  620],
       ...,
       [  14, 6191, 7045, ..., 1110,  205, 9755],
       [   0,    0,    0, ..., 3713,   14,   70],
       [   0,    0,    0, ...,   82,   93,   36]])

In [14]:
train_sequence_pad.shape

(25000, 228)

In [15]:
history = model.fit(train_sequence_pad, labels,epochs=10,batch_size=100,validation_split=0.2)

Epoch 1/10
200/200 [==============================] - 4s 15ms/step - loss: 0.5995 - acc: 0.6673 - val_loss: 0.4539 - val_acc: 0.8048
Epoch 2/10
200/200 [==============================] - 2s 10ms/step - loss: 0.2395 - acc: 0.9096 - val_loss: 0.6378 - val_acc: 0.7364
Epoch 3/10
200/200 [==============================] - 2s 10ms/step - loss: 0.1493 - acc: 0.9455 - val_loss: 0.4650 - val_acc: 0.8214
Epoch 4/10
200/200 [==============================] - 2s 10ms/step - loss: 0.0936 - acc: 0.9691 - val_loss: 0.7637 - val_acc: 0.7304
Epoch 5/10
200/200 [==============================] - 2s 10ms/step - loss: 0.0536 - acc: 0.9851 - val_loss: 0.7259 - val_acc: 0.7782
Epoch 6/10
200/200 [==============================] - 2s 12ms/step - loss: 0.0277 - acc: 0.9936 - val_loss: 0.9138 - val_acc: 0.7520
Epoch 7/10
200/200 [==============================] - 2s 10ms/step - loss: 0.0116 - acc: 0.9977 - val_loss: 1.3016 - val_acc: 0.6932
Epoch 8/10
200/200 [==============================] - 2s 12ms/step - 

# EVALUATING THE TRAINED MODEL ON THE TEXT DATA

In [16]:
data_dir=os.path.dirname('./data/aclImdb/')
test_dir = os.path.join(data_dir, 'test')
y_test=[]
test_texts=[]
for label_type in ['neg','pos']:
    dir_name=os.path.join(test_dir,label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:]=='.txt':
            file=open(os.path.join(dir_name,fname),'r',encoding='utf-8')
            read_file=file.read()
            test_texts.append(read_file)
            file.close()
            if label_type=='neg':
                y_test.append(0)
            else:
                 y_test.append(1)     


In [17]:
def preprocess__test_text(text):
    sequence=tokenizer.texts_to_sequences(text)
    test_sequence_pad=pad_sequences(sequence,maxlen=max_tokens)
    return test_sequence_pad

In [18]:
x_text=preprocess__test_text(test_texts)
x_text

array([[   0,    0,    0, ...,   32,  531,    8],
       [   4,  135,    1, ...,  176,  467,  155],
       [   0,    0,    0, ...,    8,    1,  174],
       ...,
       [ 144,  320,    4, ...,   34,  314,   38],
       [   0,    0,    0, ...,   28, 1156, 5894],
       [   0,    0,    0, ...,   58,  104, 3194]])

In [19]:
x_text.shape

(25000, 228)

In [20]:
y_test=np.array(y_test)

In [21]:
y_test.shape

(25000,)

In [22]:
loss,accu=model.evaluate(x_text,y_test)

782/782 [==============================] - 3s 3ms/step - loss: 0.8289 - acc: 0.8370


In [23]:
print("Accuracy: {0:.2%}".format(accu))

Accuracy: 83.70%


In [24]:
text1 = "This movie is fantastic! I really like it because it is so good!"
text2 = "Not a good movie!"
text3="The movie was great!"
text4 = "This movie really sucks! Can I get my money back please?"
text5="The movie was terrible..."
text6="This is a confused movie."
textss = [text1, text2, text3, text4, text5]

In [25]:
seq=preprocess__test_text(textss)

In [26]:
seq.shape

(5, 228)

In [27]:
pred=model.predict(seq)
pred=np.array(pred)
pred

array([[9.5335460e-01],
       [1.6843319e-02],
       [1.4082876e-01],
       [2.6386976e-04],
       [1.7264485e-04]], dtype=float32)

In [28]:
for text,i in zip(textss,pred):
    if i>=0.5:
        print(text,'==== >     positive review')
    else:
        print(text,'===>       negative review')

This movie is fantastic! I really like it because it is so good! ==== >     positive review
Not a good movie! ===>       negative review
The movie was great! ===>       negative review
This movie really sucks! Can I get my money back please? ===>       negative review
The movie was terrible... ===>       negative review
